In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
# TODO add periodic update heroes(using time?)
# TODO relocate dotabuff parser to another file

In [135]:
dota_stats = pd.read_csv('./dota_hero_stats.csv', index_col=0)

In [136]:
dota_stats.head()

,attack_type,id,legs,localized_name,name,primary_attr,roles
0,Melee,1,2,Anti-Mage,npc_dota_hero_antimage,agi,"['Carry', 'Escape', 'Nuker']"
1,Melee,2,2,Axe,npc_dota_hero_axe,str,"['Initiator', 'Durable', 'Disabler', 'Jungler']"
2,Ranged,3,4,Bane,npc_dota_hero_bane,int,"['Support', 'Disabler', 'Nuker', 'Durable']"
3,Melee,4,2,Bloodseeker,npc_dota_hero_bloodseeker,agi,"['Carry', 'Disabler', 'Jungler', 'Nuker', 'Ini..."
4,Ranged,5,2,Crystal Maiden,npc_dota_hero_crystal_maiden,int,"['Support', 'Disabler', 'Nuker', 'Jungler']"


In [137]:
bad_dict = {}
good_dict = {}
sum_len = dota_stats.shape[0]
perc, bord = 0, 10
for hero_name in dota_stats.localized_name:
    perc += 1
    if perc > bord:
        print(f'Proceed {round(perc/sum_len * 100, 2)}% of data')
        bord += 10
    bad_against = []
    good_against = []
    alt_hero_name = hero_name.lower().replace(' ', '-')
    alt_hero_name = alt_hero_name.replace("'", '')
    r = requests.get('https://www.dotabuff.com/heroes/{}/counters'.format(alt_hero_name),
                     headers = {'User-agent':'test'})
    # TODO add debug options
    #print(r.url, r.status_code, sep=' ')
    if r.status_code != 200: continue
    html_text = BeautifulSoup(r.text, "lxml")
    counter_table = html_text.find_all('section', {'class':'counter-outline'})
    counter_table = str(counter_table).split('"')
    for i in counter_table:
        if i[0].isupper() and len(i) < 20:
            if i not in bad_against and len(bad_against) < 5:
                bad_against.append(i)
            elif i not in good_against and i not in bad_against:
                good_against.append(i)
    #bad_against = set(bad_against)
    #good_against = set(good_against)
    bad_dict[hero_name] = bad_against
    good_dict[hero_name] = good_against
print('Done')

Proceed 9.4% of data
Proceed 17.95% of data
Proceed 26.5% of data
Proceed 35.04% of data
Proceed 43.59% of data
Proceed 52.14% of data
Proceed 60.68% of data
Proceed 69.23% of data
Proceed 77.78% of data
Proceed 86.32% of data
Proceed 94.87% of data
Done


In [138]:
dota_stats['bad_against'] = None
dota_stats['good_against'] = None

In [139]:
for hero in bad_dict:
    ind = dota_stats[dota_stats['localized_name'] == hero].index[0]
    dota_stats.at[ind, 'bad_against'] = bad_dict[hero]
    dota_stats.at[ind, 'good_against'] = good_dict[hero]

In [140]:
dota_stats.head()

,attack_type,id,legs,localized_name,name,primary_attr,roles,bad_against,good_against
0,Melee,1,2,Anti-Mage,npc_dota_hero_antimage,agi,"['Carry', 'Escape', 'Nuker']","[Meepo, Legion Commander, Terrorblade, Slardar...","[Zeus, Clockwerk, Storm Spirit, Ancient Appari..."
1,Melee,2,2,Axe,npc_dota_hero_axe,str,"['Initiator', 'Durable', 'Disabler', 'Jungler']","[Venomancer, Timbersaw, Phoenix, Spectre, Warl...","[Lycan, Broodmother, Phantom Lancer, Anti-Mage..."
2,Ranged,3,4,Bane,npc_dota_hero_bane,int,"['Support', 'Disabler', 'Nuker', 'Durable']","[Naga Siren, Meepo, Lycan, Phantom Lancer, Cha...","[Lifestealer, Phoenix, Morphling, Troll Warlor..."
3,Melee,4,2,Bloodseeker,npc_dota_hero_bloodseeker,agi,"['Carry', 'Disabler', 'Jungler', 'Nuker', 'Ini...","[Tinker, Storm Spirit, Techies, Medusa, Troll ...","[Night Stalker, Dark Seer, Razor, Pangolier, L..."
4,Ranged,5,2,Crystal Maiden,npc_dota_hero_crystal_maiden,int,"['Support', 'Disabler', 'Nuker', 'Jungler']","[Puck, Earth Spirit, Pudge, Rubick, Juggernaut]","[Phantom Lancer, Naga Siren, Meepo, Lycan, Omn..."


In [162]:
hero = input('Choose a hero:')
ind = dota_stats[dota_stats['localized_name'] == hero].index[0]
good_against = '\n\t'.join(dota_stats.at[ind, 'good_against'])
bad_against = '\n\t'.join(dota_stats.at[ind, 'bad_against'])
print('Good against:\n\t{}\nBad against:\n\t{}'.format(good_against, bad_against))
# TODO add functions
# TODO classes?
# TODO pick-helper

Choose a hero:Zeus
Good against:
	Terrorblade
	Sand King
	Naga Siren
	Bloodseeker
	Tinker
Bad against:
	Broodmother
	Anti-Mage
	Chen
	Templar Assassin
	Lycan


### Pick helper

In [167]:
# TODO heroes pick priority
# TODO try block in team choosing
my_team = []
enemy_team = []
suggest_pick = dict.fromkeys(list(dota_stats['localized_name']), 0)
while True:
    temp_pick = input('Pick a hero:')
    des = input('This hero in your team?(0-n, 1-y)')
    if des == '1':
        my_team.append(temp_pick)
    elif des == '0':
        enemy_team.append(temp_pick)
    elif des == 'q':
        break
    for hero in enemy_team:
        ind = dota_stats[dota_stats['localized_name'] == hero].index[0]
        bad_against = dota_stats.at[ind, 'bad_against']
        for enemy in bad_against:
            suggest_pick[enemy] += 1
    print(suggest_pick)
print(suggest_pick)

Pick a hero:Invoker
This hero in your team?(0-n, 1-y)0
{'Anti-Mage': 0, 'Axe': 0, 'Bane': 0, 'Bloodseeker': 0, 'Crystal Maiden': 0, 'Drow Ranger': 0, 'Earthshaker': 0, 'Juggernaut': 0, 'Mirana': 0, 'Morphling': 0, 'Shadow Fiend': 0, 'Phantom Lancer': 0, 'Puck': 1, 'Pudge': 0, 'Razor': 0, 'Sand King': 0, 'Storm Spirit': 0, 'Sven': 0, 'Tiny': 0, 'Vengeful Spirit': 0, 'Windranger': 0, 'Zeus': 0, 'Kunkka': 0, 'Lina': 0, 'Lion': 0, 'Shadow Shaman': 0, 'Slardar': 0, 'Tidehunter': 0, 'Witch Doctor': 0, 'Lich': 0, 'Riki': 0, 'Enigma': 0, 'Tinker': 0, 'Sniper': 0, 'Necrophos': 0, 'Warlock': 0, 'Beastmaster': 0, 'Queen of Pain': 0, 'Venomancer': 0, 'Faceless Void': 0, 'Wraith King': 0, 'Death Prophet': 0, 'Phantom Assassin': 0, 'Pugna': 0, 'Templar Assassin': 1, 'Viper': 0, 'Luna': 0, 'Dragon Knight': 0, 'Dazzle': 0, 'Clockwerk': 0, 'Leshrac': 0, "Nature's Prophet": 0, 'Lifestealer': 0, 'Dark Seer': 0, 'Clinkz': 0, 'Omniknight': 0, 'Enchantress': 0, 'Huskar': 0, 'Night Stalker': 0, 'Broodmother'

Pick a hero:q
This hero in your team?(0-n, 1-y)q
{'Anti-Mage': 0, 'Axe': 1, 'Bane': 0, 'Bloodseeker': 0, 'Crystal Maiden': 0, 'Drow Ranger': 0, 'Earthshaker': 0, 'Juggernaut': 0, 'Mirana': 0, 'Morphling': 0, 'Shadow Fiend': 0, 'Phantom Lancer': 2, 'Puck': 12, 'Pudge': 0, 'Razor': 0, 'Sand King': 0, 'Storm Spirit': 0, 'Sven': 0, 'Tiny': 0, 'Vengeful Spirit': 0, 'Windranger': 0, 'Zeus': 0, 'Kunkka': 0, 'Lina': 0, 'Lion': 0, 'Shadow Shaman': 0, 'Slardar': 0, 'Tidehunter': 0, 'Witch Doctor': 0, 'Lich': 0, 'Riki': 0, 'Enigma': 0, 'Tinker': 2, 'Sniper': 0, 'Necrophos': 0, 'Warlock': 0, 'Beastmaster': 0, 'Queen of Pain': 0, 'Venomancer': 0, 'Faceless Void': 0, 'Wraith King': 0, 'Death Prophet': 0, 'Phantom Assassin': 0, 'Pugna': 4, 'Templar Assassin': 5, 'Viper': 0, 'Luna': 0, 'Dragon Knight': 0, 'Dazzle': 0, 'Clockwerk': 0, 'Leshrac': 0, "Nature's Prophet": 0, 'Lifestealer': 0, 'Dark Seer': 0, 'Clinkz': 0, 'Omniknight': 0, 'Enchantress': 1, 'Huskar': 0, 'Night Stalker': 0, 'Broodmother': 8, 